# Agents
- In LangChain, agents are systems that autonomously decide how to use tools to accomplish a task. They take an input, analyze it, and choose the right tool(s) to solve the problem.
- Types of Agents in LangChain:
- from langchain.agents import AgentType

# Zero-shot Agent:

- Does not rely on prior training or a pre-defined plan.
  Decides what to do based on the current input alone.
- ZERO_SHOT_REACT_DESCRIPTION is a type of agent that performs a reasoning step before taking action. It does not rely on any chat history, meaning it makes decisions based solely on the current input. This makes it suitable for scenarios where an immediate response is required without the need for context from previous interactions.


In [77]:
#Defining Custom Tools

from langchain.agents import tool


@tool
def multiply(first_int,second_int=0) -> float:
    """Calculates Multiplication two numbers together
       Given a positive number first_int and second_int
       Multiply both the numbers with below opeator and give the result
    """
    return  first_int*second_int
@tool
def sqrt(number: float) -> float:
    """Calculates the square root of a number
       Given one positve number do the below opeeration and return result
    """
    if isinstance(number,str):
        number=float(number)
    if isinstance(number,int):
        number=float(number)
    return  number ** 0.5
 
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.invoke({"first_int":3,"second_int":5}))


print(sqrt.name)
print(sqrt.description)
print(sqrt.args)
print(sqrt.invoke({"number":625}))



multiply
Calculates Multiplication two numbers together
       Given a positive number first_int and second_int
       Multiply both the numbers with below opeator and give the result
{'first_int': {'title': 'First Int'}, 'second_int': {'default': 0, 'title': 'Second Int'}}
15
sqrt
Calculates the square root of a number
       Given one positve number do the below opeeration and return result
{'number': {'title': 'Number', 'type': 'number'}}
25.0


In [83]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents import create_openai_functions_agent
from langchain.llms import OpenAI

# Initialize the language model (OpenAI GPT)
llm = OpenAI(temperature=0)


# Create some sample tools (you can replace these with actual tools like APIs or databases)
tools = [
    
    Tool(
        name="multiply",
        func=multiply,
        description="Invoke this method only when you have two inputs in query and multiply or product of two numbers and return"
    ),
    Tool(
        name="sqrt",
        func=sqrt,
        description="Invoke this method only when you have one positive number input in query do the square root of it and return"
    )
  
]


# Initialize the zero-shot agent
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True
)


# Example task: Using the zero-shot agent to handle a question
query = "Tell me the square root of number 625"

response = agent.run(query)

print(response)



# Example task: Using the zero-shot agent to handle a question
query = "Please tell previous query"

response = agent.run(query)

print(response)






> Entering new AgentExecutor chain...
 I should use the sqrt tool to find the square root of 625
Action: sqrt
Action Input: 625
Observation: 25.0
Thought: I now know the final answer
Final Answer: 25.0

> Finished chain.
25.0


> Entering new AgentExecutor chain...
 I need to access the previous query to get the input
Action: multiply
Action Input: previous query
Observation: 
Thought: I need to access the previous query to get the input
Action: sqrt
Action Input: previous query

ValidationError: 1 validation error for sqrt
number
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='previous query', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [4]:
from langchain.agents import initialize_agent, Tool, AgentType

from langchain.agents import self_ask_with_search
from langchain.llms import OpenAI
from langchain.agents import create_openai_functions_agent
from langchain.tools import DuckDuckGoSearchResults

# Initialize the language model (e.g., OpenAI GPT)
llm = OpenAI(temperature=0)

# Set up a search tool (using DuckDuckGo search)
search_tool = DuckDuckGoSearchResults()

# Define a self-ask agent with search
tools = [
    Tool(
        name="Search",
        func=search_tool.run,  # DuckDuckGo search function
        description="Search the web for information."
    ),
]





# Initialize the Self-Ask Agent (with search)
agent = initialize_agent(tools, llm, verbose=True, agent_type="self-ask-with-search")

# Example task: ask the agent about a complex topic
query = "Who won last world cup"

# Run the agent
response = agent.run(query)

# Print the result
print(response)






> Entering new AgentExecutor chain...
 I should use the Search tool to find the answer
Action: Search
Action Input: 'Who won last world cup'

C:\Users\shanmukh.adari\AppData\Roaming\Python\Python311\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


DuckDuckGoSearchException: https://html.duckduckgo.com/html RuntimeError: error sending request for url (https://html.duckduckgo.com/html): client error (Connect)

Caused by:
    0: client error (Connect)
    1: TLS handshake failed: cert verification failed - unable to get local issuer certificate [CERTIFICATE_VERIFY_FAILED]
    2: [CERTIFICATE_VERIFY_FAILED]
    3: [CERTIFICATE_VERIFY_FAILED]

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [8]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [3]:
tools = [get_word_length]
print(tools)


[StructuredTool(name='get_word_length', description='Returns the length of a word.', args_schema=<class 'langchain_core.utils.pydantic.get_word_length'>, func=<function get_word_length at 0x0000012F38A66A20>)]


In [ ]:
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

#https://medium.com/@shravankoninti/agent-tools-basic-code-using-langchain-50e13eb07d92
# Load environment variables from .env file
load_dotenv()


# Define a very simple tool function that returns the current time
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime  # Import datetime module to get current time

    now = datetime.datetime.now()  # Get current time
    return now.strftime("%I:%M %p")  # Format time in H:MM AM/PM format


# List of tools available to the agent
tools = [
    Tool(
        name="Time",  # Name of the tool
        func=get_current_time,  # Function that the tool will execute
        # Description of the tool
        description="Useful for when you need to know the current time",
    ),
]

# Pull the prompt template from the hub
# ReAct = Reason and Action
# https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")

# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0
)

# Create the ReAct agent using the create_react_agent function
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

# Run the agent with a test query
response = agent_executor.invoke({"input": "What time is it?"})

# Print the response from the agent
print("response:", response)



> Entering new AgentExecutor chain...
I need to find out the current time.  
Action: Time  
Action Input: None  02:49 PMI now know the final answer.  
Final Answer: The current time is 02:49 PM.

> Finished chain.
response: {'input': 'What time is it?', 'output': 'The current time is 02:49 PM.'}


In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

# Define a simple tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny and warm."

tools = [
    Tool(name="Weather Info", func=get_weather, description="Provides the weather information of a city")
]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description")

# Test the agent
result = agent.run("What's the weather like in Paris?")
print(result)

C:\Users\Adari Shanmukh\AppData\Local\Temp\ipykernel_14900\2044509963.py:12: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
C:\Users\Adari Shanmukh\AppData\Local\Temp\ipykernel_14900\2044509963.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://p

The weather in Paris is sunny and warm.


In [15]:
from langchain.agents import create_conversational_agent, Tool
from langchain.llms import OpenAI

def check_order_status(order_id: str) -> str:
    return f"Order {order_id} is in transit."

tools = [
    Tool(name="Order Status Checker", func=check_order_status, description="Checks the status of an order.")
]

llm = OpenAI(temperature=0.7)
agent = create_conversational_agent(tools, llm)

# Simulating conversation
agent_memory = {}
agent.run("I would like to check the status of order #12345", memory=agent_memory)
agent.run("Can you remind me what my last query was?", memory=agent_memory)

ImportError: cannot import name 'create_conversational_agent' from 'langchain.agents' (d:\GenAI\venv\Lib\site-packages\langchain\agents\__init__.py)